Alright, listen... SCD type-2 is not that stright forward.... delta.merge actually either updates or inserts... it wont do both. so we dont operate on updates df. we take updates df, join with old df, create a new mergeKey column with nulls for new & update records and olddf.primarykey for update records. 


so now you have a df which will have new records with mergekey as NULL, update records with mergeKey as NULL and update records with mergeKey which matches with old table primary key

In [0]:
#df_new = spark.read.option('header','true').csv('dbfs:/FileStore/shared_uploads/anjaniratish@gmail.com/vegitable_priceDetails_newdata')
df_org = spark.read.option('header','true').csv('dbfs:/FileStore/shared_uploads/anjaniratish@gmail.com/vegitableFruits_priceDetails_original')

In [0]:
from pyspark.sql.functions import lit
df_org = df_org.withColumn('active',lit(1))

df_new2 = df_new2.withColumn('active',lit(1))

In [0]:
df_new2 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/anjaniratish@gmail.com/vegitable_priceDetails_newdata2-1.csv")

In [0]:
df_new2.show()

+----+------------+------+-----+------+
|item|        name|WT/QTY|price|active|
+----+------------+------+-----+------+
|  23| ground nuts|     1|   20|     1|
|  24|       peach|     1|  180|     1|
|  18|        amla|     1|   60|     1|
|  22|dragon fruit|     1|  100|     1|
+----+------------+------+-----+------+



In [0]:
df_new.show()

+----+------------+------+-----+------+
|item|        name|WT/QTY|price|active|
+----+------------+------+-----+------+
|   1|      papaya|     1|   20|     1|
|   2| pomagranate|     1|  180|     1|
|   3|  apple roya|     1|  150|     1|
|  22|dragon fruit|     1|  180|     1|
+----+------------+------+-----+------+



In [0]:
s = df_org.write.format('delta').mode('overwrite').save('/scd_2/veggieOrg')

In [0]:
orgDelta = spark.read.format('delta').load('/scd_2/veggieOrg')

In [0]:
orgDelta.show()

+----+--------------+------+-----+------+
|item|          name|WT/QTY|price|active|
+----+--------------+------+-----+------+
|   1|        papaya|     1|   20|     1|
|   2|   pomagranate|     1|  180|     1|
|   3|    apple roya|     1|  150|     1|
|  22|  dragon fruit|     1|  180|     1|
|   1|        papaya|     1|   20|     0|
|   1|        papaya|     1|   20|     0|
|  10|   brinjal p r|     1|   40|     1|
|  11|       raddish|     1|   25|     1|
|  12|        carrot|     1|   65|     1|
|  13|        ginger|     1|   55|     1|
|  14|     coccsinia|     1|   60|     1|
|  15|green chillies|     1|   50|     1|
|  16|       coconut|     1|   25|     1|
|  17|       cabbage|     1|   30|     1|
|  18|          amla|     1|   60|     1|
|  19|    green peas|     1|   40|     1|
|   2|   pomagranate|     1|  180|     0|
|   2|   pomagranate|     1|  180|     0|
|  20|     pineapple|     1|   30|     1|
|  21|  tomato local|     1|   35|     1|
+----+--------------+------+-----+

In [0]:
#figure out what are updates
updates = df_new2.alias('new').join(orgDelta.alias('old'), 'item','left').where('new.active == 1 and old.active == 1').distinct()

In [0]:
updates.show()

+----+------------+------+-----+------+------------+------+-----+------+
|item|        name|WT/QTY|price|active|        name|WT/QTY|price|active|
+----+------------+------+-----+------+------------+------+-----+------+
|  22|dragon fruit|     1|  100|     1|dragon fruit|     1|  180|     1|
|  18|        amla|     1|   60|     1|        amla|     1|   60|     1|
+----+------------+------+-----+------+------------+------+-----+------+



In [0]:
# now you know what are updates. union updates with all new data. updates will have mergekey as null, rest all will have merge key as their pk. (yes, updates will be duplicates. one recoed with merge key as null and other with merge key as pk)
stg_upds = updates.selectExpr('NULL as mergeKey', 'new.*').union(df_new2.alias('upds').filter(df_new2['active']=='1').selectExpr('upds.item as mergeKey','*'))

In [0]:
stg_upds.show()

+--------+----+------------+------+-----+------+
|mergeKey|item|        name|WT/QTY|price|active|
+--------+----+------------+------+-----+------+
|    null|  22|dragon fruit|     1|  100|     1|
|    null|  18|        amla|     1|   60|     1|
|      23|  23| ground nuts|     1|   20|     1|
|      24|  24|       peach|     1|  180|     1|
|      18|  18|        amla|     1|   60|     1|
|      22|  22|dragon fruit|     1|  100|     1|
+--------+----+------------+------+-----+------+



In [0]:
from delta.tables import *

# create delta obj
deltaObj = DeltaTable.forPath(spark, '/scd_2/veggieOrg')

In [0]:
#perform merge. we need to update matched rows as active = 0
#no need to write the results explicitly. delta takes live modifications into storage
deltaObj.alias('obj').merge(stg_upds.alias('stgUpds'), 'obj.item = stgUpds.mergeKey').whenMatchedUpdate(set={'obj.active':'0'}).whenNotMatchedInsertAll().execute()

In [0]:
test = spark.read.format('delta').load('/scd_2/veggieOrg')
test.show(100)

+----+--------------+------+-----+------+
|item|          name|WT/QTY|price|active|
+----+--------------+------+-----+------+
|  22|  dragon fruit|     1|  100|     1|
|  18|          amla|     1|   60|     1|
|   1|        papaya|     1|   20|     1|
|   1|        papaya|     1|   20|     0|
|   1|        papaya|     1|   20|     0|
|  10|   brinjal p r|     1|   40|     1|
|  11|       raddish|     1|   25|     1|
|  12|        carrot|     1|   65|     1|
|  13|        ginger|     1|   55|     1|
|  14|     coccsinia|     1|   60|     1|
|  15|green chillies|     1|   50|     1|
|  16|       coconut|     1|   25|     1|
|  17|       cabbage|     1|   30|     1|
|  18|          amla|     1|   60|     0|
|  19|    green peas|     1|   40|     1|
|   2|   pomagranate|     1|  180|     1|
|   2|   pomagranate|     1|  180|     0|
|   2|   pomagranate|     1|  180|     0|
|  20|     pineapple|     1|   30|     1|
|  21|  tomato local|     1|   35|     1|
|  22|  dragon fruit|     1|  180|

In [0]:
test.orderBy('item').show(100)

+----+--------------+------+-----+------+
|item|          name|WT/QTY|price|active|
+----+--------------+------+-----+------+
|   1|        papaya|     1|   20|     1|
|   1|        papaya|     1|   20|     0|
|   1|        papaya|     1|   20|     0|
|  10|   brinjal p r|     1|   40|     1|
|  11|       raddish|     1|   25|     1|
|  12|        carrot|     1|   65|     1|
|  13|        ginger|     1|   55|     1|
|  14|     coccsinia|     1|   60|     1|
|  15|green chillies|     1|   50|     1|
|  16|       coconut|     1|   25|     1|
|  17|       cabbage|     1|   30|     1|
|  18|          amla|     1|   60|     1|
|  18|          amla|     1|   60|     0|
|  19|    green peas|     1|   40|     1|
|   2|   pomagranate|     1|  180|     0|
|   2|   pomagranate|     1|  180|     1|
|   2|   pomagranate|     1|  180|     0|
|  20|     pineapple|     1|   30|     1|
|  21|  tomato local|     1|   35|     1|
|  22|  dragon fruit|     1|  180|     0|
|  22|  dragon fruit|     1|  100|